## Data Set

In [1]:

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize 
import folium 
import matplotlib.pyplot as plt


print('Libraries imported.')

Libraries imported.


In [34]:
CLIENT_ID = 'TZ4U5TAKOKC51EJALBKLAG1UPWLP2HSABKIQ001Y2WWC25L4' 
CLIENT_SECRET = '5COIL4Q2LO5AAUOJOXL2WONTBASRHZV0Y0RHOKER5ZNEKOPR' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: TZ4U5TAKOKC51EJALBKLAG1UPWLP2HSABKIQ001Y2WWC25L4


In [35]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["Lansing, MI", 'Grand Rapids, MI', 'Traverse City, MI', 'Royal Oak, MI', 'Ann Arbor, MI']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1d2941735") # Sushi Restaurant CATEGORY ID
    results[city] = requests.get(url).json()

In [36]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

## Data Visualization

In [76]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total Number of Sushit Restaurants in {city} = ", results[city]['response']['totalResults'])

Total Number of Sushit Restaurants in Lansing, MI =  51
Total Number of Sushit Restaurants in Grand Rapids, MI =  55
Total Number of Sushit Restaurants in Traverse City, MI =  4
Total Number of Sushit Restaurants in Royal Oak, MI =  31
Total Number of Sushit Restaurants in Ann Arbor, MI =  61


In [77]:
maps[cities[0]]

In [78]:
maps[cities[1]]

In [79]:
maps[cities[2]]

In [80]:
maps[cities[3]]

In [81]:
maps[cities[4]]

Examining the data from Foursquare we can see that there are multiple sushi restaurants located in each city. 

Total Number of Sushit Restaurants in Lansing, MI =  51
Total Number of Sushit Restaurants in Grand Rapids, MI =  55
Total Number of Sushit Restaurants in Traverse City, MI =  4
Total Number of Sushit Restaurants in Royal Oak, MI =  31
Total Number of Sushit Restaurants in Ann Arbor, MI =  61

This does not mean, however, that we automatically pick the lowest competition dense city as we also need to consider population desity. 

In [105]:
df=pd.read_csv (r'C:\Users\Maximum Carnage\Documents\LCC\Population.csv')
display(df)

,City,Population in 2010,White,African American,Asian,Other,Age 0-19,Age 20-29,Age 30-39,Age 40-49,Age 50+
0,Lansing,114297,69721,25488,4114,14985,29031,23430,16801,14515,30520
1,Grand Rapids,188040,126927,36103,4324,20686,49642,40240,27641,19368,51149
2,Traverse City,14674,13705,220,102,647,3213,1936,2010,1614,5901
3,Royal Oak,57236,40866,2152,2174,12044,9443,11962,10302,6811,18718
4,Ann Arbor,113934,80893,7405,19710,5926,25521,37142,14469,9000,27802


## Data Analysis

Reviewing the data above for population and demographics on race and age, we can see that just because Traverse City only has 4 sushi restaurants it only has a population of 14674. So we need to determine the saturation point of a sushi restaurant.



In [136]:
import math


In [153]:
LP=114297
GRP=188040
TCP=14674
ROP=57236
AAP=113934

LR=51
GRR=55
TCR=4
ROR=31
AAR=61

def round_up(n,decimals=7):
        multiplier=10**decimals
        return int(n*multiplier)/multiplier

print('Lansing Sushi Restaurant to Population Lansing:          '+ str((round_up(LR/LP))))
print('Lansing Sushi Restaurant to Population Grand Rapids:     '+ str((round_up(GRR/GRP))))
print('Lansing Sushi Restaurant to Population Traverse City:    '+ str((round_up(TCR/TCP))))
print('Lansing Sushi Restaurant to Population Royal Oak:        '+ str((round_up(ROR/ROP))))
print('Lansing Sushi Restaurant to Population Ann Arbor:        '+ str((round_up(AAR/AAP))))

Lansing Sushi Restaurant to Population Lansing:          0.0004462
Lansing Sushi Restaurant to Population Grand Rapids:     0.0002924
Lansing Sushi Restaurant to Population Traverse City:    0.0002725
Lansing Sushi Restaurant to Population Royal Oak:        0.0005416
Lansing Sushi Restaurant to Population Ann Arbor:        0.0005353


In [155]:
print('Lansing Sushi Restaurant to Population Lansing:          '+ str((round_up(LP/LR))))
print('Lansing Sushi Restaurant to Population Grand Rapids:     '+ str((round_up(GRP/GRR))))
print('Lansing Sushi Restaurant to Population Traverse City:    '+ str((round_up(TCP/TCR))))
print('Lansing Sushi Restaurant to Population Royal Oak:        '+ str((round_up(ROP/ROR))))
print('Lansing Sushi Restaurant to Population Ann Arbor:        '+ str((round_up(AAP/AAR))))

Lansing Sushi Restaurant to Population Lansing:          2241.117647
Lansing Sushi Restaurant to Population Grand Rapids:     3418.9090909
Lansing Sushi Restaurant to Population Traverse City:    3668.5
Lansing Sushi Restaurant to Population Royal Oak:        1846.3225806
Lansing Sushi Restaurant to Population Ann Arbor:        1867.7704918


## Results

Looking at the data for each city we can digest the following for each sushi restaurant there is the potential for X number of customers per city:

Lansing = 2241
Grand Rapids = 3418
Traverse City=3668
Royal Oak=1846
Ann Arbor=1867

Reviewing the data it would be the client's best option to open a starter franchise in either Traverse City or Grand Rapids because the compeition is relatively the same. One factor to keep in mind is that the Asian population according tot he 2010 Census shows that only 102 individuals living in Traverse City is of Asian decent whereas in Grand Rapdids there are 4323 individuals of Asian decent. 